In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/train.csv
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/export.pkl
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/e32a359be36d.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/e4210e7fe587.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/e93394175a19.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/0f96c358a250.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/0104b032c141.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Severe/e1fb532f55df.p

/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/356304d15a5c.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/b1c6f0997e27.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/55f7f018c61c.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/e5de79795c1d.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/1c47815f4a6b.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/5de4615a5161.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/No_DR/c6229222bf22.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/g

/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/f0c13be90519.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/fe2df69676cf.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/abdb365cacbc.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/36041171f441.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/bb45257258cc.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/a8c54e2a4b79.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/Mild/fca931da5c5e.png
/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian

In [2]:
import glob
from skimage.io import imread, imshow
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam, SGD
from keras.metrics import accuracy, Recall, Precision
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
from scikitplot.metrics import plot_cumulative_gain, plot_ks_statistic, plot_lift_curve, plot_confusion_matrix

Using TensorFlow backend.


In [3]:
train = pd.read_csv('../input/diabetic-retinopathy-224x224-gaussian-filtered/train.csv')

In [4]:
train.head(2)

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4


In [5]:
folder = '../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

In [6]:
train_generator = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                    vertical_flip=True,
                                    rotation_range = 2,
                                    validation_split=0.2)

In [7]:
train_data_generator = train_generator.flow_from_directory(folder,
                                                 target_size = (128, 128),
                                                 batch_size = 64,
                                                 class_mode = 'categorical',
                                                          subset='training')

valid_data_generator = train_generator.flow_from_directory(folder,
                                                 target_size = (128, 128),
                                                 batch_size = 64,
                                                 class_mode = 'categorical',
                                                          subset='validation')

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [12]:
cnn = Sequential()

cnn.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(128, 128, 3)))
cnn.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
cnn.add(MaxPool2D(pool_size=(2, 2)))

cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
cnn.add(MaxPool2D(pool_size=(2, 2)))
cnn.add(Dropout(0.1))

cnn.add(Flatten())
cnn.add(Dense(512, activation='relu'))
cnn.add(Dense(5, activation='softmax'))

In [13]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
cnn_model = cnn.fit_generator(train_data_generator, steps_per_epoch = 100, epochs = 10,
                  validation_data = valid_data_generator, verbose=2, shuffle=True,
                 validation_steps=200)

Epoch 1/10
 - 590s - loss: 1.3620 - accuracy: 0.5009 - val_loss: 1.2342 - val_accuracy: 0.5330
Epoch 2/10
 - 603s - loss: 1.2465 - accuracy: 0.4912 - val_loss: 1.1515 - val_accuracy: 0.4933
Epoch 3/10
 - 586s - loss: 1.2008 - accuracy: 0.5152 - val_loss: 1.0237 - val_accuracy: 0.5969
Epoch 4/10
 - 589s - loss: 1.2278 - accuracy: 0.5091 - val_loss: 1.2951 - val_accuracy: 0.4952
Epoch 5/10
 - 589s - loss: 1.0761 - accuracy: 0.5846 - val_loss: 0.9874 - val_accuracy: 0.6347
Epoch 6/10
 - 589s - loss: 0.9013 - accuracy: 0.6767 - val_loss: 0.9451 - val_accuracy: 0.6649
Epoch 7/10
 - 589s - loss: 0.8548 - accuracy: 0.6898 - val_loss: 0.7997 - val_accuracy: 0.6836
Epoch 8/10
 - 588s - loss: 0.8242 - accuracy: 0.6971 - val_loss: 0.7775 - val_accuracy: 0.6842
Epoch 9/10
 - 591s - loss: 0.8101 - accuracy: 0.7083 - val_loss: 0.7672 - val_accuracy: 0.6926
Epoch 10/10
 - 589s - loss: 0.7963 - accuracy: 0.7105 - val_loss: 0.6392 - val_accuracy: 0.7011


In [ ]:
test_accuracy = cnn.evaluate_generator(valid_data_generator, steps=624)

In [ ]:
print('The testing accuracy is :',test_accuracy[1]*100, '%')

In [ ]:
model_json = cnn.to_json()
with open("model_baseline_diabetic_retinopathy.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model_baseline_diabetic_retinopathy.h5")